## **Santander Dev Week 2025 (ETL com Python)**

- **Contexto**: Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Cientista de Dados**: Lucca de Sena Barbosa

In [4]:
import pandas as pd

### 🟡 **Opção 2: Mais Completa (Leitura de Arquivo)**

#### **Extract**

In [ ]:
users = pd.read_csv('data/clientes.csv').to_dict(orient='records')
users

[{'ID': 1,
  'Nome': 'Lucas Pereira',
  'Conta': '00012345-6',
  'Cartão': 4539876543211023,
  'Perfil_Investidor': 'Conservador'},
 {'ID': 2,
  'Nome': 'Marina Alves',
  'Conta': '00023456-7',
  'Cartão': 5198347623459081,
  'Perfil_Investidor': 'Moderado'},
 {'ID': 3,
  'Nome': 'Rafael Costa',
  'Conta': '00034567-8',
  'Cartão': 4029384756102837,
  'Perfil_Investidor': 'Conservador'},
 {'ID': 4,
  'Nome': 'Ana Beatriz',
  'Conta': '00045678-9',
  'Cartão': 4916728301947562,
  'Perfil_Investidor': 'Conservador'},
 {'ID': 5,
  'Nome': 'Bruno Silva',
  'Conta': '00056789-0',
  'Cartão': 5274019384756201,
  'Perfil_Investidor': 'Arrojado'},
 {'ID': 6,
  'Nome': 'Carla Mendes',
  'Conta': '00067890-1',
  'Cartão': 4483927561029384,
  'Perfil_Investidor': 'Conservador'},
 {'ID': 7,
  'Nome': 'Diego Rocha',
  'Conta': '00078901-2',
  'Cartão': 6011459837204568,
  'Perfil_Investidor': 'Arrojado'},
 {'ID': 8,
  'Nome': 'Fernanda Lima',
  'Conta': '00089012-3',
  'Cartão': 5321984762039457,
 

#### **Transform**

- Implementei a API do Groq utilizando o modelo ``llama-4-maverick-17b-128e-instruct`` para a geração de mensagens de marketing personalizadas. A escolha do modelo se dar por ser uma tecnologia de 2025 baseada na arquitetura MoE (128 especialistas), o que garante superioridade em lógica e precisão comparado a versões legadas. Além disso, a infraestrutura da Groq foi selecionada por oferecer acesso gratuito a modelos de alta performance, proporcionando uma alternativa economicamente viável à API da OpenAI.

- Como diferencial técnico, integrei um pipeline de RAG ao fluxo de ETL. O processo envolveu a criação de um catálogo sintético de produtos e o desenvolvimento de um script de ingestão que transforma este PDF em contexto recuperável. Ao fundamentar a geração de texto em uma base de conhecimento externa, garanto que o modelo forneça respostas coesas (grounding), alinhando a sugestão de investimento estritamente às regras de negócio de cada perfil.

In [7]:
from groq import Groq
import os
from vector_storage import load 
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
client = Groq()

In [12]:
vector_store = load()
for user in users:

    if user["Perfil_Investidor"] in ["Arrojado", "Conservador", "Moderado"]:
        retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
        docs = retriever.invoke(user["Perfil_Investidor"])

        context = ""
        for doc in docs:
            context += doc.page_content 
            context += "\n\n"
            
        context_rag = context

    else:
        context_rag = "O cliente não possui um perfil de investidor definido."

    completion = client.chat.completions.create(
        model="meta-llama/llama-4-maverick-17b-128e-instruct",
        messages=[
            {
                "role": "system",
                "content": """Você é um Especialista em Customer Experience do Santander. 
                Seu objetivo é criar mensagens curtas e engajadoras para convidar clientes a investir.
                
                Diretrizes de Marca:
                1. Tom de voz: Seguro, consultivo e próximo (sem ser informal demais).
                2. Limite: Máximo de 2 frases ou 280 caracteres (foco em notificação mobile).
                3. Proibido: Não invente dados de histórico de compras se eles não estiverem explícitos. Não prometa rentabilidade garantida.
                4. Foco: Se o cliente não tiver histórico detalhado, foque em "Construção de Futuro" ou "Reserva de Emergência"."""
            },
            {
                "role": "user",
                "content": f"""Crie uma mensagem personalizada para o seguinte cliente:
                {user}
                
                Observação: Caso o cliente não tenha perfil de investidor definido, crie uma mensagem genérica convidando-o a explorar as opções de investimento do Santander.

                A mensagem deve convidar o cliente a conhecer as opções de investimentos abaixo, baseado na busca de similaridade do perfil de investidor:


                {context_rag}

                """
            }
        ],
        temperature=0.7, 
        max_completion_tokens=200, 
        top_p=1,
        stream=True,
        stop=None
    )

    new = ""

    for chunk in completion:
        new += chunk.choices[0].delta.content or ""

    user['new'] = new   

In [15]:
new

'Olá Maurício! Como investidor arrojado, você pode estar interessado em oportunidades de alto potencial de crescimento. Conheça o Santander Future Tech Global, um fundo de ações internacionais com foco em tecnologia e inteligência artificial.'

#### **Mensagens de Marketing Geradas**:

In [17]:
from IPython.display import display, Markdown

print("="*80)
print("RELATÓRIO DE MENSAGENS GERADAS")
print("="*80 + "\n")

for user in users:
    # Recupera a última mensagem gerada (ajuste 'news[-1]' ou 'new' conforme seu código)
    mensagem = user['news'][-1] if isinstance(user.get('news'), list) else user.get('new', 'Sem mensagem')
    
    # Cria um bloco visual bonito
    display(Markdown(f"""
### 👤 Cliente: {user['Nome']}
**(ID: {user['ID']})**

---
{mensagem}
    """))
    
    # Separador visual entre clientes
    print("\n" + "—"*80 + "\n")

RELATÓRIO DE MENSAGENS GERADAS




### 👤 Cliente: Lucas Pereira
**(ID: 1)**

---
Olá, Lucas! Como cliente conservador, você valoriza a segurança e previsibilidade. Conheça nosso CDB Santander Progressivo, ideal para sua reserva de emergência, com rentabilidade que aumenta com o tempo e liquidez diária.
    


————————————————————————————————————————————————————————————————————————————————




### 👤 Cliente: Marina Alves
**(ID: 2)**

---
Olá, Marina! Como cliente Santander com perfil de investidor moderado, você pode estar interessado em conhecer o Santander Select Multimercado Macro, um fundo que busca proteger seu patrimônio e aproveitar oportunidades de mercado. Quer saber mais?
    


————————————————————————————————————————————————————————————————————————————————



...


————————————————————————————————————————————————————————————————————————————————




### 👤 Cliente: Larissa Gomes
**(ID: 14)**

---
Olá, Larissa! Como cliente conservador, você valoriza a segurança e previsibilidade. Conheça o CDB Santander Progressivo, um investimento de renda fixa com rentabilidade que aumenta com o tempo, ideal para sua reserva de emergência.
    


————————————————————————————————————————————————————————————————————————————————




### 👤 Cliente: Maurício Souza
**(ID: 15)**

---
Olá Maurício! Como investidor arrojado, você pode estar interessado em oportunidades de alto potencial de crescimento. Conheça o Santander Future Tech Global, um fundo de ações internacionais com foco em tecnologia e inteligência artificial.
    


————————————————————————————————————————————————————————————————————————————————



#### **Load**

In [ ]:
df_atualizado = pd.DataFrame(users)
df_atualizado.to_csv('data/clientes_com_marketing.csv', index=False)